In [97]:
!source activate py36

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
/usr/bin/sh: line 0: source: activate: file not found


In [98]:
import sys

#sys.path.append('/home/faske/g/anaconda3/envs/py34/lib/python3.4/site-packages')
sys.path.append('/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages')
sys.path.append("/data/gpfs/assoc/parchmanlab/tfaske/ipynb/include_utils")

import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
#import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE, call, check_output
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
#import pysam
from collections import OrderedDict, namedtuple, Counter
import operator
import multiprocessing as mp
import shutil
import tempfile
#from ipyparallel import Client
import scandir
import glob
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import pickle
import re
from itertools import chain
#import Levenshtein as lv

In [99]:
root = "/data/gpfs/assoc/denovo/tfaske/rabbruh"

In [100]:
cd $root

/data/gpfs/assoc/denovo/tfaske/rabbruh


In [101]:
pwd

'/data/gpfs/assoc/denovo/tfaske/rabbruh'

# Actual Mapping 
 

In [102]:
fastq_files = []
for seqdir in ['fastq/']:
    os.chdir('{}/{}'.format(root, seqdir))
    files = !find . -name '*fq.gz'
    files = [os.path.abspath(x) for x in files]
    for x in files:
        fastq_files.append(x)
fastq_files = sorted(fastq_files)

In [103]:
len(fastq_files),fastq_files[0]

(400, '/data/gpfs/assoc/denovo/tfaske/rabbruh/fastq/EN_TF_1.F.fq.gz')

In [104]:
cd $root

/data/gpfs/assoc/denovo/tfaske/rabbruh


In [105]:
assembly = "/data/gpfs/assoc/denovo/tfaske/rabbruh/assembly/reference.fasta"

#indexed done by dDocent 

In [109]:
# -k INT minimum seed length [19]
# -w INT band width for banded alignment [100]
# -r FLOAT look for internal seeds inside a seed longer than {-k} * FLOAT [1.5]
# -T INT minimum score to output [30]
# -R STR read group header line such as '@RG\tID:foo\tSM:bar' [null]

#@lview.remote()
def run_bwamem(args):
    import os, multiprocessing, socket #stopwatch,
    #timer = stopwatch.Timer()
    cpus = 16
    assembly, fq, outdir = args
    ID = fq.split('/')[8]
    ID = ID.split('.F.')[0]
    sam = os.path.join(outdir, "{}.sam".format(os.path.basename(ID)))
    #sam = tempfile.NamedTemporaryFile(delete=False, dir="/tmp")
    #print(fq)
    #print(ID)
    #print (sam)
    ##from command line, this works: bwa mem -k 20 -w 100 -r 1.3 -T 30 -R 
    #'@RG\tID:16_p0045_1\tLB:16_p0045_1\tSM:16_p0045_1\tPL:ILLUMINA' 
    #re_mod_map_timema_06Jun2016_RvNkF702.fasta 16_p0045_1.fastq >  aln_16_p0045_1.sam
    cmd = r"bwa mem -k 20 -w 100 -r 1.3 -T 30 -R '@RG\tID:%s\tLB:%s\tSM:%s\tPL:ILLUMINA' %s %s > %s" % (ID,ID,ID,assembly,fq,sam)
                                                               
    res = None
    res = cmd
    return  cmd 

In [110]:
bwa_dir = "/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/"
assert(bwa_dir)

In [111]:
res = []
for f in fastq_files:
    r = run_bwamem((assembly, f, bwa_dir))
    res.append(r)

In [112]:
res[0]

"bwa mem -k 20 -w 100 -r 1.3 -T 30 -R '@RG\\tID:EN_TF_1\\tLB:EN_TF_1\\tSM:EN_TF_1\\tPL:ILLUMINA' /data/gpfs/assoc/denovo/tfaske/rabbruh/assembly/reference.fasta /data/gpfs/assoc/denovo/tfaske/rabbruh/fastq/EN_TF_1.F.fq.gz > /data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_1.sam"

In [113]:
pwd

'/data/gpfs/assoc/denovo/tfaske/rabbruh'

In [198]:
### select options
account = 'cpu-s5-denovo-0'
partition = 'cpu-core-0'
jobname = 'rabbruh'
time = '4-00:00:00' #time limit 4 days
cpus = 64
mem_cpu = 2500
email = 'tfaske@nevada.unr.edu'

In [205]:
def write_bwamem_sh(account,partition,jobname,time,cpus,mem_cpu,email,cmds):
    #print(account)
    #print(partition)
    #print(cpus)
    #print(email)
    #print(cmds[0])
    with open("SNPcall/run_bwamem.sh", "w") as o:
        o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name %s_bwamem
#SBATCH --output output_bwamem.txt
#SBATCH --mail-type=BEGIN,END
#SBATCH --mail-user=%s \n\n""" % (account,partition,time,int(cpus),int(mem_cpu),jobname,email))
        for elem in cmds:
            o.write("%s \n" % (elem))

In [206]:
cd $root

/data/gpfs/assoc/denovo/tfaske/rabbruh


In [207]:
write_bwamem_sh(account,partition,jobname,time,cpus,mem_cpu,email,res)

# Run run_bwamem.sh
    cd /data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall 
    source activate py36
    sbatch run_bwamem.sh

# run_bwamem timed out
    write new shell script for ones that didn't get completed

In [128]:
fqs = !find /data/gpfs/assoc/denovo/tfaske/rabbruh/fastq/ -name "*fq.gz"

In [129]:
sams = !find $bwa_dir -name "*.sam"

In [130]:
len(fqs), len(sams)

(400, 131)

In [131]:
print(fqs[0])
print(sams[0])

/data/gpfs/assoc/denovo/tfaske/rabbruh/fastq/EN_TF_67.F.fq.gz
/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_159.sam


In [134]:
fq_ID = [f.split('/')[8].split('.F.')[0] for f in fqs]
sam_ID = [s.split('/')[9].split('.sam')[0] for s in sams]

In [135]:
print(fq_ID[0])
print(sam_ID[0])

EN_TF_67
EN_TF_159


In [150]:
fq_sam_diff = list(set(fq_ID).difference(sam_ID))
len(fq_sam_diff), len(fqs) - len(sams)

(269, 269)

In [149]:
fq_sam_diff = list(set(fq_ID).difference(sam_ID))

In [163]:
fastq_redo = []
for f in fq_sam_diff:
    f = f + '.F.fq.gz'
    os.chdir('{}/{}'.format(root, 'fastq'))
    files = !find . -name $f
    files = [os.path.abspath(x) for x in files]
    for x in files:
        fastq_redo.append(x)
fastq_redo = sorted(fastq_redo)

In [169]:
print(len(fastq_redo),fastq_redo[0])
assert len(fastq_redo) == len(fq_sam_diff) == len(fqs) - len(sams)

269 /data/gpfs/assoc/denovo/tfaske/rabbruh/fastq/EN_TF_217.F.fq.gz


In [170]:
res_redo = []
for f in fastq_redo:
    r = run_bwamem((assembly, f, bwa_dir))
    res_redo.append(r)

In [172]:
len(res_redo),res_redo[0]

(269,
 "bwa mem -k 20 -w 100 -r 1.3 -T 30 -R '@RG\\tID:EN_TF_217\\tLB:EN_TF_217\\tSM:EN_TF_217\\tPL:ILLUMINA' /data/gpfs/assoc/denovo/tfaske/rabbruh/assembly/reference.fasta /data/gpfs/assoc/denovo/tfaske/rabbruh/fastq/EN_TF_217.F.fq.gz > /data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_217.sam")

In [244]:
### select options
account = 'cpu-s1-bionres-0'
partition = 'cpu-s1-bionres-0'
jobname = 'rabbruh'
time = '4-00:00:00' #time limit 4 days
cpus = 30
mem_cpu = 2500
email = 'tfaske@nevada.unr.edu'

In [245]:
def write_bwamem_redo_sh(account,partition,jobname,time,cpus,mem_cpu,email,cmds):
    #print(account)
    #print(partition)
    #print(cpus)
    #print(email)
    #print(cmds[0])
    with open("SNPcall/run_bwamem_bionres.sh", "w") as o:
        o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name %s_bwamem
#SBATCH --output output_bwamem_redo.txt
#SBATCH --mail-type=BEGIN,END
#SBATCH --mail-user=%s \n\n""" % (account,partition,time,int(cpus),int(mem_cpu),jobname,email))
        for elem in cmds:
            o.write("%s \n" % (elem))

In [246]:
cd $root

/data/gpfs/assoc/denovo/tfaske/rabbruh


In [247]:
write_bwamem_redo_sh(account,partition,jobname,time,cpus,mem_cpu,email,res_redo)

# Write sh for each file and then bash to submit file

In [223]:
cd $root

/data/gpfs/assoc/denovo/tfaske/rabbruh


In [222]:
!mkdir shbwa

In [210]:
print(len(fastq_redo),fastq_redo[0])

269 /data/gpfs/assoc/denovo/tfaske/rabbruh/fastq/EN_TF_217.F.fq.gz


In [211]:
res_redo[0]

"bwa mem -k 20 -w 100 -r 1.3 -T 30 -R '@RG\\tID:EN_TF_217\\tLB:EN_TF_217\\tSM:EN_TF_217\\tPL:ILLUMINA' /data/gpfs/assoc/denovo/tfaske/rabbruh/assembly/reference.fasta /data/gpfs/assoc/denovo/tfaske/rabbruh/fastq/EN_TF_217.F.fq.gz > /data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa/EN_TF_217.sam"

In [212]:
fq_ID = [f.split('/')[8].split('.F.')[0] for f in fastq_redo]
print(len(fq_ID),fq_ID[0])

269 EN_TF_217


In [213]:
### select options
account = 'cpu-s5-denovo-0'
partition = 'cpu-core-0'
jobname = 'rabbruh'
cpus = 1
mem_cpu = 2500
email = 'tfaske@nevada.unr.edu'

In [230]:
def write_bwamem_ind_sh(account,partition,jobname,cpus,mem_cpu,email,fq_ID,cmds):
    #print(account)
    #print(partition)
    #print(cpus)
    #print(email)
    #print(cmds[0])
    for i in range(0,len(cmds)):
        with open("SNPcall/shbwa/run_bwamem_%s.sh" % (fq_ID[i]), "w") as o:
            o.write("""#!/usr/bin/env bash
    #SBATCH --account=%s
    #SBATCH --partition=%s
    #SBATCH --ntasks 1
    #SBATCH --cpus-per-task %d
    #SBATCH --mem-per-cpu=%d
    #SBATCH --job-name %s_bwamem
    #SBATCH --output output_bwamem_%s.txt
    #SBATCH --mail-user=%s \n\n
    
    %s \n""" % (account,partition,int(cpus),int(mem_cpu),fq_ID,fq_ID,email,cmds[i]))

In [231]:
write_bwamem_ind_sh(account,partition,jobname,cpus,mem_cpu,email,fq_ID,res_redo)

In [232]:
#find all sh and write bash script
shbwa_files = []
os.chdir('{}/{}'.format(root, 'SNPcall/shbwa'))
files = !find . -name '*.sh'
files = [os.path.abspath(x) for x in files]
for x in files:
        shbwa_files.append(x)
shbwa_files = sorted(shbwa_files)

In [233]:
len(shbwa_files),shbwa_files[0]

(269,
 '/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/shbwa/run_bwamem_EN_TF_217.sh')

In [237]:
def write_bash_bwamem_sh(sh_files):
    with open("SNPcall/run_bash_bwa.sh", "w") as o:
        o.write("""#!/usr/bin/env bash \n\n""")
        for f in sh_files:
            o.write("sbatch %s \n" % (f))    

In [238]:
cd $root

/data/gpfs/assoc/denovo/tfaske/rabbruh


In [239]:
 write_bash_bwamem_sh(shbwa_files)

# Run run_bash_bwamem_sh
    cd /data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall 
    source activate py36
    bash run_bash_bwa.sh

# sam2bam

In [250]:
fqs = !find /data/gpfs/assoc/denovo/tfaske/rabbruh/fastq/ -name "*fq.gz"
sams = !find $bwa_dir -name "*.sam"

In [251]:
len(fqs), len(sams)

(400, 400)

In [252]:
SNPcall_dir = "/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/"
assert(SNPcall_dir)

In [253]:
cd $SNPcall_dir

/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall


In [254]:
### select options
account = 'cpu-s1-bionres-0'
partition = 'cpu-s1-bionres-0'
jobname = 'rabbruh'
time = '4-00:00:00' #time limit 4 days
cpus = 30
mem_cpu = 2500
email = 'tfaske@nevada.unr.edu'

In [261]:
def write_sam2bam_sh(account,partition,cpus,mem_cpu,jobname,email,sams):
    with open("./run_sam2bam.sh", "w") as o:
        o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name %s_sam2bam
#SBATCH --output output_sam2bam.txt
#SBATCH --mail-type=BEGIN,END
#SBATCH --mail-user=%s \n\n""" % (account,partition,int(cpus),int(mem_cpu),jobname,email))
        for s in sams:
            bam = s.replace(".sam", ".bam")
            bam_sorted = "%s_sorted.bam" % bam.replace(".bam", "")
            o.write("samtools view -b {0} -o {1}\nsamtools sort -@ {3} {1} -o {2}\nsamtools index {2}\n".format(s,bam,bam_sorted,cpus))

In [262]:
write_sam2bam_sh(account,partition,cpus,mem_cpu,jobname,email,sams)

# run_sam2bam.sh
    cd /data/gpfs/home/tfaske/d/rabbruh/SNPcall 
    source activate py36
    sbatch run_sam2bam.sh


In [263]:
cd $bwa_dir

/data/gpfs/assoc/denovo/tfaske/rabbruh/SNPcall/bwa


In [268]:
sorted_bams = []
files = !find . -type f -name '*sorted.bam'
files = [os.path.abspath(x) for x in files if 'bam' in x]
for x in files:
    sorted_bams.append(x)
sorted_bams = sorted(sorted_bams)

#assert len(sorted_bams) == len(fastq_files)

len(sorted_bams), len(fqs), len(sams)
#missing 3 bam files from lane1

(400, 400, 400)